In [72]:
import os 
import pandas as pd
from collections import defaultdict
import numpy as np 

Step 1. Get the all sample experiement longlist: 
Use cut and unique in bash - (could be done in the notebook) 



Step 2. Get the metadata and peptides: 
Done in bash with the command below 

Step 3. Get the formating done within this notebook 


# Preprocess the annotation 

In [2]:
ann_path = '/cluster/work/grlab/projects/projects2020_OHSU/annotation/gencode.v32.annotation.gtf'

In [3]:
# From Immunopepper preprocess.py 
def attribute_item_to_dict(a_item, file_type, feature_type):
    """  From attribute item in annotation file to get corresponding dictionary

    Parameters
    ----------
    a_item: str. attribute item
    file_type: str. Choose from {'gtf', 'gff', 'gff3'}
    feature_type: str. Extract other fields. We only
        consider 'CDS', 'mRNA' and 'transcript'

    Returns
    -------
    gtf_dict: dict. store all the necessary data

    """    
    
    gtf_dict = {}
    if file_type.lower() == 'gtf':
        attribute_list = a_item.split('; ')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split(' ')
            gtf_dict[pair[0]] = pair[1][1:-1]
    elif file_type.lower() == 'gff3':
        attribute_list = a_item.split(';')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split('=')
            gtf_dict[pair[0]] = pair[1]
    elif file_type.lower() == 'gff':
        gff_dict = {}
        attribute_list = a_item.split(';')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split('=')
            gff_dict[pair[0]] = pair[1]  # delete "", currently now work on level 2
        if feature_type == 'CDS':
            gtf_dict['transcript_id'] = gff_dict['Parent']
        elif feature_type in {'mRNA', 'transcript'}:  # mRNA or transcript
            gtf_dict['gene_id'] = gff_dict['geneID']
            gtf_dict['transcript_id'] = gff_dict['ID']
            gtf_dict['gene_type'] = gff_dict['gene_type']
            gtf_dict['transcript_type'] = gff_dict['transcript_type']

    return gtf_dict

In [4]:
#  From Immunopepper preprocess.py 
def leq_strand(coord1, coord2, strand):
    if strand == "+":
        return coord1 <= coord2
    else:
        return coord1 >= coord2


In [25]:
# From Immunopepper preprocess.py 
transcript_to_gene_dict = {}    # transcript -> gene id


gene_to_transcript_dict = {}    # gene_id -> list of transcripts
gene_cds_begin_dict = {}        # gene -> list of first CDS exons

transcript_to_cds_dict = {}     # transcript -> list of CDS exons
transcript_cds_begin_dict = {}  # transcript -> first exon of the CDS
transcript_to_strand = {}

file_type = ann_path.split('.')[-1]
chromesome_set = set()
# collect information from annotation file
for line in open(ann_path, 'r'):
    if line[0] == '#':
        continue
    item = line.strip().split('\t')
    chromesome_set.add(item[0])
    feature_type = item[2]
    attribute_item = item[-1]
    attribute_dict = attribute_item_to_dict(attribute_item, file_type, feature_type)
    # store relationship between gene ID and its transcript IDs
    if feature_type in ['transcript', 'mRNA']:
        gene_id = attribute_dict['gene_id']
        transcript_id = attribute_dict['transcript_id']
        if attribute_dict['gene_type'] != 'protein_coding' or attribute_dict['transcript_type']  != 'protein_coding':
            continue
        assert (transcript_id not in transcript_to_gene_dict)
        transcript_to_gene_dict[transcript_id] = gene_id
        if gene_id in gene_to_transcript_dict and transcript_id not in gene_to_transcript_dict[gene_id]:
            gene_to_transcript_dict[gene_id].append(transcript_id)
        else:
            gene_to_transcript_dict[gene_id] = [transcript_id]
        # Todo python is 0-based while gene annotation file(.gtf, .vcf, .maf) is one based
    elif feature_type == "CDS":
        parent_ts = attribute_dict['transcript_id']
        strand_mode = item[6]
        cds_left = int(item[3])-1
        cds_right = int(item[4])
        frameshift = int(item[7])
        transcript_to_strand[parent_ts] = strand_mode
        if parent_ts in transcript_to_cds_dict:
            transcript_to_cds_dict[parent_ts].append((cds_left, cds_right, frameshift))
        else:
            transcript_to_cds_dict[parent_ts] = [(cds_left, cds_right, frameshift)]
        if strand_mode == "+" :
            cds_start, cds_stop = cds_left, cds_right
        else:
            cds_start, cds_stop = cds_right, cds_left

        # we only consider the start of the whole CoDing Segment
        if parent_ts not in transcript_cds_begin_dict or \
           leq_strand(cds_start, transcript_cds_begin_dict[parent_ts][0], strand_mode):
            transcript_cds_begin_dict[parent_ts] = (cds_start, cds_stop, item)

# collect first CDS exons for all transcripts of a gene
for ts_key in transcript_to_gene_dict:
    target_gene = transcript_to_gene_dict[ts_key]
    if target_gene not in gene_cds_begin_dict:
        gene_cds_begin_dict[target_gene] = []
    if ts_key in transcript_cds_begin_dict:
        gene_cds_begin_dict[target_gene].append(transcript_cds_begin_dict[ts_key])


In [39]:
transcript_cds_begin_dict_bis = {}
transcript_cds_end_dict_bis = {}

gene_cds_begin_dict_bis = defaultdict(list)
gene_cds_end_dict_bis = defaultdict(list)

# will be in reading order 
for ts_key in transcript_to_cds_dict:
    if transcript_to_strand[ts_key] == '+': # '+'
        transcript_cds_begin_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][0][0],
                                                 transcript_to_cds_dict[ts_key][0][1], '+')
        transcript_cds_end_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][-1][0],
                                                 transcript_to_cds_dict[ts_key][0][1], '+')
        

    else: 
        transcript_cds_begin_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][0][1],
                                                 transcript_to_cds_dict[ts_key][0][0], '-')
        transcript_cds_end_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][-1][1],
                                                 transcript_to_cds_dict[ts_key][0][0], '-')
    
    assert(transcript_cds_begin_dict_bis[ts_key][0] == transcript_cds_begin_dict[ts_key][0])
    assert(transcript_cds_begin_dict_bis[ts_key][1] == transcript_cds_begin_dict[ts_key][1])

# collect first, last CDS exons for all transcripts of a gene
for ts_key in transcript_to_gene_dict:
    target_gene = transcript_to_gene_dict[ts_key]
    gene_cds_begin_dict_bis[target_gene].append(transcript_cds_begin_dict_bis[ts_key])
    gene_cds_end_dict_bis[target_gene].append(transcript_cds_end_dict_bis[ts_key])

In [40]:
gene_cds_begin_dict_bis

defaultdict(list,
            {'ENSG00000186092.6': [(65564, 65573, '+'), (69090, 70005, '+')],
             'ENSG00000284733.1': [(451678, 450742, '-')],
             'ENSG00000284662.1': [(686654, 685718, '-')],
             'ENSG00000187634.12': [(924431, 924948, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (925941, 926013, '+'),
              (930311, 930336, '+'),
              (939274, 939460, '+')],
             'ENSG00000188976.11': [(959240, 959214, '-')],
             'ENSG00000187961.14': [(960693, 960800, '+'),
              (962706, 962917, '+')],
             'ENSG00000187583.11': [(966531, 966614, '+'),
              (966531, 966614, '+'),
              (966531, 966614

# Format the peptide raw file (from step 2)

In [41]:
file_meta = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/TCGA-A2-A0D2-01A-21R-A034-07.all_d4aee54_GTEXcore_kmer_peptides_raw.tsv'

In [45]:
with open(file_meta, 'r') as fp:
    lines = fp.readlines()
    for line in lines[0:23]:
        print(line)

AAAALQQHS,ALGVQQPSLLGASPTIYTQQTALAAAGLTTQTPANYQLTQTAALQQQAAAAAAALQQHSSVFSQGWLQNYMIRLDLWMKMYSF,ENSG00000060339.14:27_45_49:0:68736877:3-exons_9-mer,0,ENSG00000060339.14,chr10,+,ref,0,0,nan,0,nan,nan,68736877;68737048;68742427;68742430;68742487;68742562,68736875;68737048;68742427;68742430;68742487;68742564,"(27, 45, 49)",3-exons_9-mer ALGVQQPSLLGASPTIYTQQTALAAAGLTTQTPANYQLTQTAALQQQAAAAAAALQQHSSVFSQGWLQNYMIRLDLWMKMYSFSL,ENSG00000060339.14:27_45_50:0:68736877:3-exons_9-mer,0,ENSG00000060339.14,chr10,+,ref,0,0,nan,0,nan,nan,68736877;68737048;68742427;68742430;68742487;68742568,68736875;68737048;68742427;68742430;68742487;68742569,"(27, 45, 50)",3-exons_9-mer

AAADGVLSV,WLRGELDRAAADGVLSVYPGLPASPSAGSACDQHLLELHWLCGGLCLRP,ENSG00000119333.11:70_64:0:128640727:2-exons,1,ENSG00000119333.11,chr9,-,ref,1,0,nan,0,nan,nan,128640690;128640727;128636917;128637027;None;None,128640690;128640729;128636917;128637027;None;None,"(70, 64)",2-exons

AAADGWTMG,WLRGELDRAAADGWTMGTGARLSPSCVPGTWTGETCVPSSRRPWWRSPALSC

In [90]:
line = line.replace('3-exons_9-mer ', '3-exons_9-mer@').replace('2-exons ', '2-exons@')
kmer = line.split(',')[0]
peptides = ','.join(line.split(',')[1:])
peptides = peptides.split('@')
for pep in peptides:
    pep = pep.split(',')
    pep_seq = pep[0]
    pep_gene = pep[3]
    pep_orig_coord = pep[13]
    pep_modif_coord = pep[14]
    strand = pep[4]
    assert('/' not in pep_orig_coord)
    if strand == '+':
        start_cds = [ first_exon[0] for first_exon in gene_cds_begin_dict_bis[pep_gene] ] 
        pep_start = np.int(pep_orig_coord.split(';')[0])
        if pep_start in start_cds:
            pep_5include = 1
        else: 
            pep_5include = 0 
    
    

In [89]:
pep

['MEDPSGAREPRARPRERDPGRRPHPDQGRTHDRPRDRPGDPRRKRSSDGNRRRDGDRDPERDQERDGNRDRNRDRERERERERDPDRGPRRDTHRDAGPRAGEHGVWEKPRQSRTRDGARGLTWDAAADPKVNPLRRDICPRPPGLDERRWNITSQRRKDSWNATNANTCALGEVSRFAGCLIIYCHWWFWKGWSELVKQKFKQIPLWVPSIFISVLVRFLFS',
 'ENSG00000140832.9:11_15_21:0:71626229:3-exons_9-mer/ENSG00000140832.9:2_15_21:0:71626229:3-exons_9-mer/ENSG00000140832.9:7_15_21:0:71626229:3-exons_9-mer',
 '0',
 'ENSG00000140832.9',
 'chr16',
 '+',
 'ref',
 '1',
 '1',
 'nan',
 '0',
 'nan',
 'nan',
 '71626229;71626607;71626680;71626696;71629366;71629680',
 '71626160;71626607;71626680;71626696;71629366;71629680/71626186;71626607;71626680;71626696;71629366;71629680/71626185;71626607;71626680;71626696;71629366;71629680',
 '"(2',
 ' 15',
 ' 21)/(11',
 ' 15',
 ' 21)/(7',
 ' 15',
 ' 21)"',
 '3-exons_9-mer\n']

In [84]:
pep_start

71626229

In [85]:
start_cds

[71626229, 71626229, 71626229, 71626229, 71629459]

In [86]:
pep_5include

1

In [88]:
pep_orig_coord

'71626229;71626607;71626680;71626696;71629366;71629680'

In [87]:
gene_cds_end_dict_bis[pep_gene]

[(71629366, 71626696, '+'),
 (71640389, 71626533, '+'),
 (71634192, 71626696, '+'),
 (71640389, 71626696, '+'),
 (71634192, 71629494, '+')]

In [67]:
for ts in gene_to_transcript_dict[pep_gene]:
    print(ts)
    print(transcript_to_cds_dict[ts])

ENST00000567566.1
[(71626229, 71626696, 0), (71629366, 71629556, 1)]
ENST00000565261.1
[(71626229, 71626533, 0), (71629366, 71629494, 2), (71640389, 71640401, 0)]
ENST00000268485.7
[(71626229, 71626696, 0), (71629366, 71629494, 1), (71634192, 71634800, 2)]
ENST00000299952.4
[(71626229, 71626696, 0), (71629366, 71629494, 1), (71640389, 71641024, 2)]
ENST00000567501.1
[(71629459, 71629494, 0), (71634192, 71634337, 1)]


In [63]:
pep_orig_coord

'71626229;71626607;71626680;71626696;71629366;71629680'

In [65]:
pep_modif_coord.split('/')

['71626160;71626607;71626680;71626696;71629366;71629680',
 '71626186;71626607;71626680;71626696;71629366;71629680',
 '71626185;71626607;71626680;71626696;71629366;71629680']

In [57]:
pep

['MEDPSGAREPRARPRERDPGRRPHPDQGRTHDRPRDRPGDPRRKRSSDGNRRRDGDRDPERDQERDGNRDRNRDRERERERERDPDRGPRRDTHRDAGPRAGEHGVWEKPRQSRTRDGARGLTWDAAADPKVNPLRRDICPRPPGLDERRWNITSQRRKDSWNATNANTCALGEVSRFAGCLIIYCHWWFWKGWSELVKQKFKQIPLWVPSIFISVLVRFLFS',
 'ENSG00000140832.9:11_15_21:0:71626229:3-exons_9-mer/ENSG00000140832.9:2_15_21:0:71626229:3-exons_9-mer/ENSG00000140832.9:7_15_21:0:71626229:3-exons_9-mer',
 '0',
 'ENSG00000140832.9',
 'chr16',
 '+',
 'ref',
 '1',
 '1',
 'nan',
 '0',
 'nan',
 'nan',
 '71626229;71626607;71626680;71626696;71629366;71629680',
 '71626160;71626607;71626680;71626696;71629366;71629680/71626186;71626607;71626680;71626696;71629366;71629680/71626185;71626607;71626680;71626696;71629366;71629680',
 '"(2',
 ' 15',
 ' 21)/(11',
 ' 15',
 ' 21)/(7',
 ' 15',
 ' 21)"',
 '3-exons_9-mer\n']

In [53]:
line = lines[21]
kmer = line[0]
